In [1]:
import os
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import time

In [3]:
raw_data = pd.read_csv('../data/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [4]:
batch_size = 1024
is_cuda = torch.cuda.is_available()
# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("training on GPU")
else:
    device = torch.device("cpu")
    print("training on CPU")

training on CPU


In [5]:
class Data(Dataset):
    def __init__(self, df, window_size=1000, sequence_len=batch_size):
        self.rows = df.shape[0] // (window_size*sequence_len)
        self.data, self.labels = [], []
        
        for s in range(self.rows):
            seg = df.iloc[s*window_size*sequence_len: (s+1)*window_size*sequence_len]
            x = seg.acoustic_data.values
            y = seg.time_to_failure.values[-1]
            self.data.append(create_X(x))
            self.labels.append(y)
            
    def __len__(self):
        return self.rows
    
    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.data[idx].astype(np.float32)),
            self.labels[idx]
        )


def feature_extraction(time_step):
    return np.c_[time_step.mean(axis=1), 
                 np.percentile(np.abs(time_step), q=[0, 25, 50, 75, 100], axis=1).T,
                 time_step.std(axis=1)]



def create_X(x, window_size=1000, seq_len=batch_size):
    X = x.reshape(seq_len, -1)
    return np.c_[feature_extraction(X),
                 feature_extraction(X[:, -window_size // 10:]),]

In [6]:
class GRU(nn.Module):
    
    def __init__(self, D_in, H, D_out, n_layers, dropout = 0.2):
        super(GRU, self).__init__()
        self.hidden_size = H
        self.n_layers = n_layers

        self.gru = nn.GRU(input_size = D_in, hidden_size = H, num_layers = n_layers, dropout=dropout)
        self.fc = nn.Linear(H, D_out)
        self.relu = nn.ReLU()
    
    def forward(self, x, h):
        out, h = self.gru(x,h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h

    def init_hidden(self, batch_size):
        is_cuda = torch.cuda.is_available()
        if is_cuda:
            device = torch.device("cuda")
        else:
            device = torch.device("cpu")
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_size).zero_().to(device)
        print(hidden.shape)
        return hidden

In [14]:
def train_model(train_loader, lr = 0.01, hidden_dim = 256, epochs = 10):
    input_size = next(iter(train_loader))[0].shape[2]
    output_size = 1
    n_layers = 2

    model = GRU(input_size, hidden_dim, output_size, n_layers)

    model.to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    epoch_times = []

    print('=========> Starting training')
    for epoch in range(1, epochs + 1):
        start_time = time.time()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0

        for x, label in train_loader:
            counter += 1
            h = h.data
            model.zero_grad()

            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())

            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter % 100 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))

        end_time = time.time()

        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, epochs, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(start_time-start_time)))
        epoch_times.append(start_time-start_time)
        
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

In [1]:
def evaluate(model, data_loader):
    h = model.init_hidden(batch_size)
    out = []
    targets = []
    model.eval()
    diff = []
    with torch.no_grad():
        for x, y in data_loader:
            h = h.data
            out, h = model(x.to(device).float(), h)
            out = out.reshape(-1)
            print(out.shape)
            diff.append(y - out)
            
    print(diff[0])
    MAE = abs(sum(diff))/len(diff)
    
    print("MAE: {}%".format(MAE*100))
    return MAE

In [26]:
mask = np.random.rand(len(raw_data)) < 0.8
train = raw_data[mask]
test = raw_data[~mask]

train_set = Data(train)
test_set = Data(test)
train_loader = DataLoader(train_set,shuffle=False, batch_size=batch_size)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size)

In [15]:
model = train_model(train_loader)

=========> Starting training
torch.Size([2, 1024, 256])
Epoch 1/10 Done, Total Loss: 45.798465728759766
Total Time Elapsed: 0.0 seconds
torch.Size([2, 1024, 256])
Epoch 2/10 Done, Total Loss: 19.654050827026367
Total Time Elapsed: 0.0 seconds
torch.Size([2, 1024, 256])
Epoch 3/10 Done, Total Loss: 13.838205337524414
Total Time Elapsed: 0.0 seconds
torch.Size([2, 1024, 256])
Epoch 4/10 Done, Total Loss: 13.35927677154541
Total Time Elapsed: 0.0 seconds
torch.Size([2, 1024, 256])
Epoch 5/10 Done, Total Loss: 14.092580795288086
Total Time Elapsed: 0.0 seconds
torch.Size([2, 1024, 256])
Epoch 6/10 Done, Total Loss: 14.69754695892334
Total Time Elapsed: 0.0 seconds
torch.Size([2, 1024, 256])
Epoch 7/10 Done, Total Loss: 14.61975383758545
Total Time Elapsed: 0.0 seconds
torch.Size([2, 1024, 256])
Epoch 8/10 Done, Total Loss: 14.003288269042969
Total Time Elapsed: 0.0 seconds
torch.Size([2, 1024, 256])
Epoch 9/10 Done, Total Loss: 13.516972541809082
Total Time Elapsed: 0.0 seconds
torch.Size(

In [2]:
gru_MAE = evaluate(model, test_loader)

NameError: name 'model' is not defined